# Pitch detection

Compare well-known pitch with YIN estimate.

YIN seems to fail on contrabassoon and trombone samples with octave errors...

In [1]:
import sys
sys.path.append('..')

In [2]:
import warnings
warnings.simplefilter('ignore', UserWarning)

In [3]:
import IPython.display as display
import librosa
import matplotlib.pyplot as plot
import numpy as np

from philharmonia import dataframe, datafile

In [4]:
def yin(sample):

    x, sr = librosa.load(datafile(sample.file), sr=None, mono=True)
    lengths=dict(frame_length=x.size*2, win_length=x.size, hop_length=x.size)
    results = librosa.yin(x, fmin=sample.pitch/2, fmax=sample.pitch*2, sr=sr, **lengths)

    return round(results[-1], 2)

def error(x, y):

    return '<--' if abs(y / x - 1) > 0.1 else ''

In [5]:
query = [
    "percussion==False",
    "note == 'C'",
    "style == 'normal'",
    "length == '1'",
]

filter = ['file', 'instrument', 'note', 'octave', 'dynamic', 'pitch']
sort = ['pitch', 'dynamic']

data = dataframe().query(' & '.join(query)).filter(filter).sort_values(sort)

data['yin'] = data.apply(lambda sample: yin(sample), axis=1)
data['octave error'] = data.apply(lambda sample: error(sample.pitch, sample.yin), axis=1)

In [6]:
display.HTML(data.to_html())

,file,instrument,note,octave,dynamic,pitch,yin,octave error
7994,contrabassoon/contrabassoon_C1_1_forte_normal.mp3,contrabassoon,C,1.0,forte,32.70,32.86,
8166,contrabassoon/contrabassoon_C1_1_fortissimo_normal.mp3,contrabassoon,C,1.0,fortissimo,32.70,32.77,
8263,contrabassoon/contrabassoon_C1_1_mezzo-forte_normal.mp3,contrabassoon,C,1.0,mezzo-forte,32.70,32.81,
8191,contrabassoon/contrabassoon_C1_1_piano_normal.mp3,contrabassoon,C,1.0,piano,32.70,32.90,
4580,bassoon/bassoon_C2_1_forte_normal.mp3,bassoon,C,2.0,forte,65.41,66.18,
7914,contrabassoon/contrabassoon_C2_1_forte_normal.mp3,contrabassoon,C,2.0,forte,65.41,65.69,
10554,tuba/tuba_C2_1_forte_normal.mp3,tuba,C,2.0,forte,65.41,65.01,
4345,bassoon/bassoon_C2_1_fortissimo_normal.mp3,bassoon,C,2.0,fortissimo,65.41,66.17,
10593,tuba/tuba_C2_1_fortissimo_normal.mp3,tuba,C,2.0,fortissimo,65.41,61.64,
11742,french horn/french-horn_C2_1_fortissimo_normal.mp3,french-horn,C,2.0,fortissimo,65.41,66.17,
